In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
import time

In [ ]:
options = webdriver.ChromeOptions()
# Using SOCKS Proxy (Optional)
options.add_argument("--proxy-server=socks5://127.0.0.1:8080")
browser = webdriver.Chrome('f:/utils/chromedriver/chromedriver.exe', options=chrome_options)
browser.get("https://www.commerce.alaska.gov/cbp/main/search/entities")

In [ ]:
entity = browser.find_element_by_id("EntityName")
entity.clear()
entity.send_keys("son")
search = browser.find_element_by_id("search")
search.click()

In [ ]:
df = None
page = 1
while True:
    spage = Select(browser.find_element_by_css_selector(".pgNumTxt select"))
    try:
        print('page = %d' % page)
        spage.select_by_visible_text(str(page))
        page += 1
    except Exception as e:
        print(e)
        break
    time.sleep(5)
    dfs = pd.read_html(browser.page_source, attrs={'class': 'deptGridView'})
    if df is None:
        df = dfs[0]
    else:
        df = pd.concat([df, dfs[0]])

In [ ]:
page = 102
while True:
    print('page = %d' % page)
    # FIXME: Get the data over 2,000 result using direct URL
    browser.get('https://www.commerce.alaska.gov/cbp/main/Search/EntitiesResults?EntityName=son&IsStartsWithSearch=False&CurrentOnly=False&pageNumber=%d' % page)
    time.sleep(5)
    try:
        dfs = pd.read_html(browser.page_source, attrs={'class': 'deptGridView'})
        df = pd.concat([df, dfs[0]])
    except Exception as e:
        print(e)
        break
    page += 1


In [ ]:
df

In [ ]:
df.reset_index(drop=True).to_csv('son.csv', index=False)

In [ ]:
browser.quit()